Uses push model concept (also works :D)

In [1]:
try: import simplejson as json
except ImportError: import json

import gzip,codecs,numpy as np,random,copy
import scipy.optimize as opt

In [2]:
#with open("finefood_train_random.json","r") as infile:
#with open("beeradvocate_train_random.json","r") as infile:
#with open("beeradvocate_train_lastrating.json","r") as infile:
#with open("finefood_train_lastrating.json","r") as infile:
#with open("ratebeer_train_lastrating.json","r") as infile:
with open("ratebeer_train_random.json","r") as infile:
    train = json.load(infile)
infile.close()
#with open("finefood_test_random.json","r") as infile:
#with open("beeradvocate_test_random.json","r") as infile:
#with open("beeradvocate_test_lastrating.json","r") as infile:
#with open("finefood_test_lastrating.json","r") as infile:
#with open("ratebeer_test_lastrating.json","r") as infile:
with open("ratebeer_test_random.json","r") as infile:
    test = json.load(infile)
infile.close()
#with open("finefood_quickmap_random.json","r") as infile:
#with open("beeradvocate_quickmap_random.json","r") as infile:
#with open("beeradvocate_quickmap_lastrating.json","r") as infile:
#with open("finefood_quickmap_lastrating.json","r") as infile:
#with open("ratebeer_quickmap_lastrating.json","r") as infile:
with open("ratebeer_quickmap_random.json","r") as infile:
    quickmap = json.load(infile)
infile.close()

print(len(train),len(test),len(quickmap))
train = sorted(train, key = lambda k : int(k["review/time"]))

2704815 4760 2709575


In [3]:
Iu = dict() #set of products reviewed by users
Ui = dict() #set of users who reviewed the product

In [4]:
for review in train:
        item = review["product/productId"]
        user = review["review/userId"]
        if item in Ui:
            Ui[item].append(user)
        else:
            Ui[item] = [user]
        if user in Iu:
            Iu[user].append(item)
        else:
            Iu[user] = [item]

In [5]:
print(len(Iu))

4760


In [6]:
distinct_user_set = set()
distinct_item_set = set()
for review in train:
    if review["review/userId"] not in distinct_user_set:
        distinct_user_set.add(review["review/userId"])
    if review["product/productId"] not in distinct_item_set:
        distinct_item_set.add(review["product/productId"])
print(len(distinct_user_set), len(distinct_item_set))

4760 110032


In [7]:
import sys
sys.setrecursionlimit(20000)

In [8]:
#with open("finefood_user_map_random.json",'r') as infile:
#with open("finefood_user_map_lastrating.json",'r') as infile:
#with open("beeradvocate_user_map_random.json",'r') as infile:
#with open("beeradvocate_user_map_lastrating.json",'r') as infile:
#with open("ratebeer_user_map_lastrating.json",'r') as infile:
with open("ratebeer_user_map_random.json",'r') as infile:
        user_map = json.load(infile)
infile.close()
#with open("finefood_item_map_random.json",'r') as infile:
#with open("finefood_item_map_lastrating.json",'r') as infile:
#with open("beeradvocate_item_map_random.json",'r') as infile:
#with open("beeradvocate_item_map_lastrating.json",'r') as infile:
#with open("ratebeer_item_map_lastrating.json",'r') as infile:
with open("ratebeer_item_map_random.json",'r') as infile:
        item_map = json.load(infile)
infile.close()

user_map_int = {}
for key in user_map:
    user_map_int[int(key)] = user_map[key]
item_map_int = {}
for key in item_map:
    item_map_int[int(key)] = item_map[key]

# Expertise modelling

In [12]:
class ExpertiseLFM(object):
    ''' Expertise LFM class implements the evolution latent factor model of collaborative filtering 
    using matrix factorization
    '''
    
    def __init__(self,train_data, Iu_reg, Ui_reg, userproduct_dict,userset,itemset,\
                 usermap, itemmap,k,Lambda1,Lambda2,E,mode):
        ''' requires Iu and Ui matrix information, quick mapping of reviews to (user,product),
         k =number of latent factor dimensions, 
         lambda1 = reg parameter, lambda2 = smoothing parameter,
         E = number of experience levels.
        '''
        
        self.Ntrain = len(train_data)               #Number of training samples
        self.train_data = train_data                #training data 
        self.Iu = self.deepish_copy(Iu_reg)         #Iu mapping  
        self.Ui = self.deepish_copy(Ui_reg)         #Ui mapping
        self.quickmap = userproduct_dict            #uses key as (userid-itemid) for quick mapping to required review
        self.user_set = userset
        self.item_set = itemset
        
        self.user_map = usermap     #mapping for easier transformation from long gradient vector to individual gradients
        self.item_map = itemmap
        
        
        #hyperparameters
        self.Lambda1 = Lambda1    #regularization param
        self.Lambda2 = Lambda2    #smoothing reg param     
        self.k = k                # number of latent factor dimension (low dimensional repr)
        self.E = E                #number of experience levels
        self.mode = mode
        self.final_param = self.init_theta()  #current final_parameters
        self.init_exp()
        
    
    
    def init_theta(self):
        ''' Initializes the parameters of E recommender models
        
        flat_theta = <alpha_G, Bu_G, Bi_G, alpha_e1..E, Bu_e1..E, Bi_e1..E, Gu_e1...E, Gi_e1...E>
        '''
        flat_theta = []
        
        rating_arr = [review["review/score"] for review in self.train_data]
        avg_rating = np.mean(rating_arr)
        self.alpha_G = avg_rating                                 #global offset
        self.Bu_G = dict()                                        #user bias (global)
        self.Bi_G = dict()                                        #item bias (global)
        
        for i in range(len(self.user_map)):
            self.Bu_G[self.user_map[i]] = np.random.random(1).item() 
        for i in range(len(self.item_map)):
            self.Bi_G[self.item_map[i]] = np.random.random(1).item() 
        
        flat_theta.append(self.alpha_G)
        flat_theta.extend(list(self.Bu_G.values()))
        flat_theta.extend(list(self.Bi_G.values()))
        
        self.alpha = np.random.rand(self.E)                     #individual offset parameters per exp
        self.Bu = [dict() for i in range(self.E)]               #user bias per exp
        self.Bi = [dict() for i in range(self.E)]               #item bias per exp
        self.Gu = [dict() for i in range(self.E)]               #user latent factor vector repr per exp
        self.Gi = [dict() for i in range(self.E)]               #item latent factor vector repr per exp
        
        flat_theta.extend(self.alpha)
        for e in range(self.E):
            for i in range(len(self.user_map)):
                self.Bu[e][self.user_map[i]] = np.random.random(1).item()
                flat_theta.append(self.Bu[e][self.user_map[i]]) 
        for e in range(self.E):        
            for j in range(len(self.item_map)):
                self.Bi[e][self.item_map[j]] = np.random.random(1).item() 
                flat_theta.append(self.Bi[e][self.item_map[j]])
        for e in range(self.E):
            for i in range(len(self.user_map)):
                self.Gu[e][self.user_map[i]] = np.random.uniform(0,1,(1,self.k))
                flat_theta.extend(np.array(list(self.Gu[e][self.user_map[i]])).flatten())
        for e in range(self.E):        
            for j in range(len(self.item_map)):
                self.Gi[e][self.item_map[j]] = np.random.uniform(0,1,(1,self.k))
                flat_theta.extend(np.array(list(self.Gi[e][self.item_map[j]])).flatten())
        
        self.recparam = (1 + len(self.user_set) + len(self.item_set) \
                            + self.k*(len(self.user_set) + len(self.item_set))) #per experience level parameters
        self.globalparam = 1 + len(self.user_set) + len(self.item_set)   #global parameters
        self.totalparams = self.recparam * self.E + self.globalparam
        return np.array(flat_theta)
    
    def init_exp(self):
        ''' Initializes experience for each user-item combination uniformly over rating time'''
        self.eui = dict()  #experience dictionary of dictionaries (1: user level, 2: item level)
        if self.mode == 1 or self.mode==2: # Community 
            num_items_in_level = round(len(self.train_data)/self.E)
            if num_items_in_level ==0:
                print("Something went wrong")
            else:
                cur_level =0
                for i in range(len(self.train_data)):
                    review = self.train_data[i] 
                    user = review["review/userId"]
                    item = review["product/productId"]
                    if user not in self.eui:
                        self.eui[user]= {}
                    if i!=0 and i% num_items_in_level == 0 and cur_level != self.E-1:
                        cur_level +=1
                    self.eui[user][item] = cur_level
        else:                        #user level    
            for user in self.user_set:
                self.eui[user] = {}
                num_items_in_level = round(len(self.Iu[user])/self.E)
                if num_items_in_level ==0:
                    print("Something went wrong.", len(self.Iu[user]),user)
                cur_level = 0
                for i in range(len(self.Iu[user])):
                    if i!=0 and i% num_items_in_level == 0 and cur_level != self.E-1:
                         cur_level+=1
                    item = self.Iu[user][i]
                    self.eui[user][item] = cur_level
        print("Experience assignment done")
                
    def OPT_rec(self,i,j,n,user):
        '''
         i = current experience level, j = jth rating of user, n = number of ratings given by user
         internally modelled as experience 0 to E-1 (so experience E is invalid)
        '''
        if i==self.E or j==n:
            return np.inf
        elif self.OPT[i,j] >=0: #intial value = -1
            return self.OPT[i,j]
        else:
            item = self.Iu[user][j]  # jth rating
            rating = self.quickmap[user+"-"+item]["review/score"]
            temp  = min(self.OPT_rec(i+1,j+1,n,user), self.OPT_rec(i,j+1,n,user))
            rec_e = self.pred_e(user,item,i)   #current level rating
            if temp == np.inf:
                self.OPT[i,j] = (rec_e - rating)**2 
            else:
                self.OPT[i,j] = (rec_e - rating)**2 + temp
                  
            return self.OPT[i,j]
    
        
    def assign_exp_level_iterative(self,user):
        ''' Iterative solution for assigning experience to each user'''
        n = len(self.Iu[user])
        #compute the last column values i.e. the last item's values for each exp level
        item = self.Iu[user][n-1]
        for i in range(self.E):
            rating = self.quickmap[user+"-"+item]["review/score"]
            rec_e = self.pred_e(user,item,i)   #current level rating
            self.OPT[i,n-1] = (rec_e - rating)**2
        # now compute the upper most level row values (i.e. all items's highest exp level)
        for j in range(n-2,-1,-1):
            item = self.Iu[user][j]
            rating = self.quickmap[user+"-"+item]["review/score"]
            rec_e = self.pred_e(user,item,self.E-1)   #current level rating
            self.OPT[self.E-1,j] = (rec_e - rating)**2 + self.OPT[self.E-1,j+1]
        #now update every other value in the matrix
        for j in range(n-2,-1,-1):
            item = self.Iu[user][j]
            for i in range(self.E-2,-1,-1):
                rating = self.quickmap[user+"-"+item]["review/score"]
                rec_e = self.pred_e(user,item,i)   #current level rating
                temp  = min(self.OPT[i+1,j+1],self.OPT[i,j+1])
                self.OPT[i,j] = (rec_e - rating)**2 + temp
    
    def assign_exp_level(self):
        ''' Using the DP solution similar to Longest Common SubSequence, predict new experience level
        for each user-item combination'''
        k = 0
        count=0
        for user in self.Iu:
            n = len(self.Iu[user])
            self.OPT = np.matrix([[-1.0]*n]*self.E)  #initialize to invalid values
            #recursive solution
            #for i in range(self.E):
            #    self.OPT_rec(i,0,n,user)
            #Iterative solution:
            self.assign_exp_level_iterative(user)
            cur_level = np.argmin(self.OPT[:,0])
            j = 0
            item = self.Iu[user][j]
            self.eui[user][item] = cur_level
            start_level = cur_level
            j+=1
            while (j < n):
                try: 
                    if cur_level != self.E-1 and self.OPT[cur_level,j] >= self.OPT[cur_level+1,j]:
                        cur_level +=1
                    item = self.Iu[user][j]
                    if cur_level != self.eui[user][item]:
                        count+=1
                    self.eui[user][item] = cur_level
                    j+=1
                except Exception as e:
                    print(e.args,i,j,n)
            if k%1000==0:
                print("user: {} start level: {} end level: {}".format(user,start_level,cur_level))
            k+=1 
        print("Number of experience levels changed:{}".format(count))
        return count
    
    
    def assign_exp_community(self):
        print("Changing community experience levels")
        n = self.Ntrain
        self.OPT = np.matrix([[-1.0]*n]*self.E)
        count = 0 
        #first get the last column values (i.e. the last review of the community)
        review = self.train_data[-1]
        user = review["review/userId"]
        item = review["product/productId"]
        rating = review["review/score"]
        for i in range(self.E):
            rec_e = self.pred_e(user,item,i)   #current level rating
            self.OPT[i,n-1] = (rec_e - rating)**2
        # now compute the upper most level row values (i.e. all items's highest exp level)
        for j in range(n-2,-1,-1):
            review = self.train_data[j]
            user = review["review/userId"]
            item = review["product/productId"]
            rating = review["review/score"]
            rec_e = self.pred_e(user,item,self.E-1)   #current level rating
            self.OPT[self.E-1,j] = (rec_e - rating)**2 + self.OPT[self.E-1,j+1]
        #now update every other value in the matrix
        for j in range(n-2,-1,-1):
            review = self.train_data[j]
            user = review["review/userId"]
            item = review["product/productId"]
            for i in range(self.E-2,-1,-1):
                rating = review["review/score"]
                rec_e = self.pred_e(user,item,i)   #current level rating
                temp  = min(self.OPT[i+1,j+1],self.OPT[i,j+1])
                self.OPT[i,j] = (rec_e - rating)**2 + temp
        
        cur_level = np.argmin(self.OPT[:,0])
        j = 0
        review = self.train_data[j]
        user = review["review/userId"]
        item = review["product/productId"]
        self.eui[user][item] = cur_level
        start_level = cur_level
        j+=1
        while (j < n):
            try: 
                if cur_level != self.E-1 and self.OPT[cur_level,j] >= self.OPT[cur_level+1,j]:
                        cur_level +=1
                review = self.train_data[j]
                user = review["review/userId"]
                item = review["product/productId"]
                if cur_level != self.eui[user][item]:
                    count+=1
                self.eui[user][item] = cur_level
                j+=1
            except Exception as e:
                print(e.args,i,j,n)
            if j%100000 ==0:
                print(user,item, self.eui[user][item])
        return count
    
    def retrieve_theta_components(self,theta):
        ''' Sets all parameters from the long theta vector obtained after update rule'''
        j = 0
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        self.alpha_G = theta[j]
        j+=1
        for i in range(umap_len):
            self.Bu_G[self.user_map[i]] = theta[j]
            j+=1
        for i in range(imap_len):
            self.Bi_G[self.item_map[i]] = theta[j]
            j+=1 
        for e in range(self.E):
            self.alpha[e] = theta[j]
            j+=1   
        for e in range(self.E):
            for i in range(umap_len):
                self.Bu[e][self.user_map[i]] = theta[j]
                j+=1
        for e in range(self.E):
            for i in range(imap_len):
                self.Bi[e][self.item_map[i]] = theta[j]
                j+=1 
        for e in range(self.E):
            for i in range(umap_len):
                self.Gu[e][self.user_map[i]] = np.array(theta[j:j+self.k])
                j+=self.k  
        for e in range(self.E):
            for i in range(imap_len):
                self.Gi[e][self.item_map[i]] = np.array(theta[j:j+self.k])
                j+=self.k   
        if j!= len(theta):
            print("Something went wrong. Not all theta values were used")
            
    
    def pred_e(self,user,item,e):
        return self.alpha_G + self.Bu_G[user] + self.Bi_G[item] +\
                      self.alpha[e] + self.Bu[e][user] + self.Bi[e][item] +\
                      np.asscalar(np.dot(self.Gu[e][user], self.Gi[e][item].T))
    
    def f(self,theta):
        '''Calculates the value of the objective function (loss) on the training data. '''
        self.retrieve_theta_components(theta)
        #mean squared error
        error = 0
        for review in self.train_data:
            user = review['review/userId']
            item = review["product/productId"]
            e = self.eui[user][item]
            error += (self.pred_e(user,item,e) - review["review/score"])**2
        error /= self.Ntrain
        #regularization terms
        reg_complexity = 0
        #ignore global values for now in regularization
        Bu_np = np.array(list(self.Bu_G.values()))
        Bi_np = np.array(list(self.Bi_G.values()))
        reg_complexity = np.sum(np.square(Bu_np)) + np.sum(np.square(Bi_np))
        for e in range(self.E):
            reg_complexity += np.square(self.alpha[e])
            Bu_np = np.array(list(self.Bu[e].values()))
            Bi_np = np.array(list(self.Bi[e].values()))
            reg_complexity += np.sum(np.square(Bu_np)) + np.sum(np.square(Bi_np))
            for user in self.Gu[e]:
                reg_complexity += np.linalg.norm(self.Gu[e][user])**2
            for item in self.Gi[e]:
                reg_complexity += np.linalg.norm(self.Gi[e][item])**2
        #regularization (smoothing cost)
        reg_term = 0
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        for e in range(1,self.E):
            reg_term += (self.alpha[e-1] - self.alpha[e])**2
        for e in range(1,self.E):
            for i in range(umap_len):
                reg_term += (self.Bu[e-1][self.user_map[i]] - self.Bu[e][self.user_map[i]])**2
        for e in range(1,self.E):
            for i in range(imap_len):
                reg_term += (self.Bi[e-1][self.item_map[i]] - self.Bi[e][self.item_map[i]])**2       
        for e in range(1,self.E):
            for i in range(umap_len):
                reg_term += np.linalg.norm(self.Gu[e-1][self.user_map[i]] - self.Gu[e][self.user_map[i]])**2  
        for e in range(1,self.E):
            for i in range(imap_len):
                reg_term += np.linalg.norm(self.Gi[e-1][self.item_map[i]] - self.Gi[e][self.item_map[i]])**2
        return (error + self.Lambda1* reg_complexity +  self.Lambda2 * reg_term)*0.5
    
    def fprime_one_func(self,theta):
        ''' does all gradient work in one function. Should be definitely faster'''
        self.retrieve_theta_components(theta)
        flat_gradient = []
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        
        self.alpha_G_grad = 0                              
        self.Bu_G_grad = dict()                                      
        self.Bi_G_grad = dict()                                     
        
        for i in range(len(self.user_map)):
            self.Bu_G_grad[self.user_map[i]] = 0.0
        for i in range(len(self.item_map)):
            self.Bi_G_grad[self.item_map[i]] = 0.0
        
        self.alpha_grad = np.zeros(self.E)                     #individual offset parameters per exp
        self.Bu_grad = [dict() for i in range(self.E)]               #user bias per exp
        self.Bi_grad = [dict() for i in range(self.E)]               #item bias per exp
        self.Gu_grad = [dict() for i in range(self.E)]               #user latent factor vector repr per exp
        self.Gi_grad = [dict() for i in range(self.E)]               #item latent factor vector repr per exp
        
        for e in range(self.E):
            for i in range(len(self.user_map)):
                self.Bu_grad[e][self.user_map[i]] = 0.0
                self.Gu_grad[e][self.user_map[i]] = np.zeros((1,self.k))
            for j in range(len(self.item_map)):
                self.Bi_grad[e][self.item_map[j]] = 0.0
                self.Gi_grad[e][self.item_map[j]] = np.zeros((1,self.k))
        
        for review in self.train_data:
            user = review['review/userId']
            item = review["product/productId"]
            e = self.eui[user][item]
            rat_diff = self.pred_e(user,item,e)- review["review/score"] 
            rat_diff/= self.Ntrain
            self.alpha_G_grad += rat_diff
            self.Bu_G_grad[user] += rat_diff
            self.Bi_G_grad[item] += rat_diff
            self.alpha_grad[e] += rat_diff
            self.Bu_grad[e][user] += rat_diff
            self.Bi_grad[e][item] += rat_diff
            self.Gu_grad[e][user] += rat_diff * self.Gi[e][item]
            self.Gi_grad[e][item] += rat_diff * self.Gu[e][user]
        
        for i in range(len(self.user_map)):
            user = self.user_map[i]
            self.Bu_G_grad[user] += self.Lambda1 * self.Bu_G[user]
        for j in range(len(self.item_map)):
            item = self.item_map[j]
            self.Bi_G_grad[item] += self.Lambda1 * self.Bi_G[item]
        
        for e in range(self.E):
            self.alpha_grad[e] += self.Lambda1*self.alpha[e]
            if e == self.E-1:
                self.alpha_grad[e]  += self.Lambda2 * (self.alpha[e] - self.alpha[e-1])
            elif e == 0:
                self.alpha_grad[e]  += self.Lambda2 * (self.alpha[e] - self.alpha[e+1])
            else:
                self.alpha_grad[e]  += self.Lambda2 * (2*self.alpha[e] - self.alpha[e-1]  - self.alpha[e+1])
            
            for i in range(len(self.user_map)):
                user = self.user_map[i]
                self.Bu_grad[e][user] += self.Lambda1*self.Bu[e][user]
                self.Gu_grad[e][user] += self.Lambda1*self.Gu[e][user]
                if e == self.E-1:
                    self.Bu_grad[e][user] += self.Lambda2* (self.Bu[e][user] - self.Bu[e-1][user])
                    self.Gu_grad[e][user] += self.Lambda2* (self.Gu[e][user] - self.Gu[e-1][user]) 
                elif e==0:
                    self.Bu_grad[e][user] += self.Lambda2* (self.Bu[e][user] - self.Bu[e+1][user])
                    self.Gu_grad[e][user] += self.Lambda2* (self.Gu[e][user] - self.Gu[e+1][user]) 
                else:
                    self.Bu_grad[e][user] += self.Lambda2* (2*self.Bu[e][user] - self.Bu[e-1][user] \
                                                            - self.Bu[e+1][user])
                    self.Gu_grad[e][user] += self.Lambda2* (2*self.Gu[e][user] - self.Gu[e-1][user] \
                                                            - self.Gu[e+1][user])
        
            for j in range(len(self.item_map)):
                item = self.item_map[j]
                self.Bi_grad[e][item] += self.Lambda1*self.Bi[e][item]
                self.Gi_grad[e][item] += self.Lambda1*self.Gi[e][item]
                if e == self.E-1:
                    self.Bi_grad[e][item] += self.Lambda2* (self.Bi[e][item] - self.Bi[e-1][item])
                    self.Gi_grad[e][item] += self.Lambda2* (self.Gi[e][item] - self.Gi[e-1][item]) 
                elif e==0:
                    self.Bi_grad[e][item] += self.Lambda2* (self.Bi[e][item] - self.Bi[e+1][item])
                    self.Gi_grad[e][item] += self.Lambda2* (self.Gi[e][item] - self.Gi[e+1][item]) 
                else:
                    self.Bi_grad[e][item] += self.Lambda2* (2*self.Bi[e][item] - self.Bi[e-1][item] \
                                                            - self.Bi[e+1][item])
                    self.Gi_grad[e][item] += self.Lambda2* (2*self.Gi[e][item] - self.Gi[e-1][item] \
                                                            - self.Gi[e+1][item])
        
        #compute gradient wrt global parameters
        flat_gradient.append(self.alpha_G_grad)
        flat_gradient.extend(list(self.Bu_G_grad.values()))
        flat_gradient.extend(list(self.Bi_G_grad.values()))
        #compute gradient wrt experience parameters
        flat_gradient.extend(self.alpha_grad)
        for e in range(self.E): 
            flat_gradient.extend(list(self.Bu_grad[e].values()))
        for e in range(self.E):
            flat_gradient.extend(list(self.Bi_grad[e].values()))
        for e in range(self.E):
            flat_gradient.extend(np.array(list(self.Gu_grad[e].values())).flatten())
        for e in range(self.E):
            flat_gradient.extend(np.array(list(self.Gi_grad[e].values())).flatten())
        return np.array(flat_gradient)
    
    def call(self,theta):
        print("{} Objective value: {}".format(self.i, self.f(theta)))
        self.i+=1
    
    def objectiveloss_lbfgs(self,thetaguess, grad_tolerance):
        self.i =0;
        flat_theta_guess = thetaguess
        flat_theta,value,d = opt.fmin_l_bfgs_b(self.f,flat_theta_guess,self.fprime_one_func,\
                                              disp=True,\
                                              maxiter = 30, callback = self.call, iprint=0)
        #set the final parameters to the final value returned by fmin_l_bfgs_b
        return flat_theta
    
    
    def push_model(self):
        ''' push the model towards more regularized place'''
        e_alpha_avg = np.mean(self.alpha)
        self.alpha_G += e_alpha_avg
        self.alpha -= e_alpha_avg
        
        for user in self.Bu_G:
            e_Bu_avg = 0 
            for e in range(self.E):
                e_Bu_avg += self.Bu[e][user]
            e_Bu_avg /= self.E
            self.Bu_G[user] += e_Bu_avg
            for e in range(self.E):
                self.Bu[e][user] -= e_Bu_avg
        for item in self.Bi_G:
            e_Bi_avg = 0
            for e in range(self.E):
                e_Bi_avg += self.Bi[e][item]
            e_Bi_avg /= self.E
            self.Bi_G[item] += e_Bi_avg
            for e in range(self.E):
                self.Bi[e][item] -= e_Bi_avg
    
    def als (self,grad_tolerance):
        ''' bad name. not exactly ALS, but performs LBFGS gradient descent, and sets experience level'''
        guess = self.final_param
        for m in range(10):
            print("Iteration {}:".format(m+1))
            print("Objective function value: {}".format(self.f(guess)))
            guess = self.objectiveloss_lbfgs(guess, grad_tolerance)
            self.final_param = guess.copy()
            self.retrieve_theta_components(guess)
            print("Model alpha parameters(before push): ",[a + self.alpha_G for a in self.alpha])
            self.push_model()
            print("Model alpha parameters: ",[a + self.alpha_G for a in self.alpha])
            if self.mode ==2:  #community learned
                count = self.assign_exp_community()
            elif self.mode==4: #user learned
                count = self.assign_exp_level()
            print("Objective function value: {}".format(self.f(guess)))
            if count==0:
                print("Breaking")
                return
            
    def mse_test(self,test_data):
        ''' Uses Mean Squared Error as evaluation metric on test data provided by user'''
        self.retrieve_theta_components(self.final_param)
        error = 0
        unknown_data_count =0;
        for review in test_data:
            user = review["review/userId"]
            item = review["product/productId"]
            #assign nearest experience to user-item combo
            rtime = int(review["review/time"])
            time_arr = []
            for it in self.Iu[user]:
                time_arr.append(int(self.quickmap[user+"-"+it]["review/time"]))
            if all(time_arr[i] <= time_arr[i+1] for i in range(len(time_arr)-1))==False:
                print("raising error. Something went wrong. List should be sorted by default")
            index = np.searchsorted(time_arr,rtime)
            if index == len(self.Iu[user]):
                closest_it = self.Iu[user][index-1]
            else:
                closest_it = self.Iu[user][index]
            
            e = self.eui[user][closest_it]
            try:
                error += (self.pred_e(user,item,e) - review["review/score"])**2
            except Exception as e:
                print(e)
                unknown_data_count+=1
        
        if unknown_data_count>0:
            print("Warning! Unknown {} new data rows; Incorporating this into MSE".format(unknown_data_count))
        return error / (len(test_data) - unknown_data_count)
    
    def fprime(self, theta):
        ''' Calculates the gradient of objective function f()'''
        self.retrieve_theta_components(theta)
        flat_gradient = []
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        #compute gradient wrt global parameters
        flat_gradient.append(self.compute_gradient_wrt_alpha_global())
        Bu_grad = self.compute_gradient_wrt_Bu_global()
        Bi_grad = self.compute_gradient_wrt_Bi_global()
        flat_gradient.extend(list(Bu_grad.values()))
        flat_gradient.extend(list(Bi_grad.values()))
        #compute gradient wrt experience parameters
        for e in range(self.E):
            flat_gradient.append(self.compute_gradient_wrt_alpha(e))
        for e in range(self.E): 
            Bu_grad = self.compute_gradient_wrt_Bu(e)
            flat_gradient.extend(list(Bu_grad.values()))
        for e in range(self.E):
            Bi_grad = self.compute_gradient_wrt_Bi(e)
            flat_gradient.extend(list(Bi_grad.values()))
        for e in range(self.E):
            Gu_grad = self.compute_gradient_wrt_Gu(e)
            flat_gradient.extend(np.array(list(Gu_grad.values())).flatten())
        for e in range(self.E):
            Gi_grad = self.compute_gradient_wrt_Gi(e)
            flat_gradient.extend(np.array(list(Gi_grad.values())).flatten())
        return np.array(flat_gradient)
    
    def vanilla_gd(self,eta,guess):
        self.i =0;
        flat_theta_guess = guess
        for i in range(100):
            flat_gradient = self.fprime(flat_theta_guess)
            flat_theta_guess -= eta*flat_gradient
            if i%50 ==0: print("{} U : Objective value: {}".format(i,self.f(flat_theta_guess)))
            self.i +=1
        return flat_theta_guess
    
    def vanilla_als (self,eta):
        guess = self.init_theta()
        #print("param = ",guess[self.recparam])
        for m in range(100):
            guess = self.vanilla_gd(eta, guess)
            self.final_param = guess
            self.retrieve_theta_components(guess)
            count = self.assign_exp_level()
            if count==0:
                print("Breaking")
                return
   
    def compute_gradient_wrt_alpha_global(self):
        tempsum = 0
        for review in self.train_data:  #each user item id combo
            user = review['review/userId']
            item = review["product/productId"]
            e = self.eui[user][item]
            tempsum += ( self.pred_e(user,item,e)- review["review/score"])
        tempsum /= self.Ntrain
        return tempsum
        
    def compute_gradient_wrt_Bu_global(self):
        Bu_grad = {}
        for user in self.Bu_G: 
            total = 0.0
            for item in self.Iu[user]:
                e = self.eui[user][item]
                total += ( self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            total += self.Lambda1*self.Bu_G[user]
            Bu_grad[user] = total
        return Bu_grad
    
    def compute_gradient_wrt_Bi_global(self):
        Bi_grad = {}
        for item in self.Bi_G:
            total = 0.0
            for user in self.Ui[item]:
                e = self.eui[user][item]
                total += ( self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            total += self.Lambda1*self.Bi_G[item]
            Bi_grad[item] = total
        return Bi_grad
    
    def compute_gradient_wrt_alpha(self,exp):
        ''' Compute gradient of objective with respect to alpha parameter of given experience exp level'''
        tempsum = 0
        for review in self.train_data:  #each user item id combo
            user = review['review/userId']
            item = review["product/productId"]
            e = self.eui[user][item]
            if e == exp:  #only take the values pertaining the current level
                tempsum += ( self.pred_e(user,item,e) - review["review/score"]) 
        
        tempsum /= self.Ntrain
        
        #regularization term
        tempsum += self.Lambda1*self.alpha[exp]
        
        if exp == self.E-1:
            tempsum += self.Lambda2 * (self.alpha[exp] - self.alpha[exp-1])
        elif exp == 0:
            tempsum += self.Lambda2 * (self.alpha[exp] - self.alpha[exp+1])
        else:
            tempsum += self.Lambda2 * (2*self.alpha[exp] - self.alpha[exp-1]  - self.alpha[exp+1])
        return tempsum
    
    def compute_gradient_wrt_Bu(self,e):
        ''' Compute gradient of objective with respect to Bu parameter'''
        Bu_grad = {}
        for user in self.Bu[e]: 
            total = 0.0
            for item in self.Iu[user]:
                if self.eui[user][item] == e:
                    total += ( self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            total += self.Lambda1*self.Bu[e][user]
            if e == self.E-1:
                total += self.Lambda2* (self.Bu[e][user] - self.Bu[e-1][user])
            elif e==0:
                total += self.Lambda2* (self.Bu[e][user] - self.Bu[e+1][user])
            else:
                total += self.Lambda2* (2*self.Bu[e][user] - self.Bu[e-1][user]  - self.Bu[e+1][user])
            Bu_grad[user] = total
        return Bu_grad
    
    def compute_gradient_wrt_Bi(self,e):
        ''' Compute gradient of objective with respect to Bi parameter'''
        Bi_grad = {}
        for item in self.Bi[e]:
            total = 0.0
            for user in self.Ui[item]:
                if self.eui[user][item] == e:
                    total +=  (self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            total += self.Lambda1*self.Bi[e][item]
            if e == self.E-1:
                total += self.Lambda2* (self.Bi[e][item] - self.Bi[e-1][item])
            elif e==0:
                total += self.Lambda2* (self.Bi[e][item] - self.Bi[e+1][item])
            else:
                total += self.Lambda2* (2*self.Bi[e][item] - self.Bi[e-1][item]  - self.Bi[e+1][item])
            Bi_grad[item] = total
        return Bi_grad
    
    def compute_gradient_wrt_Gu(self,e):
        ''' Compute gradient of objective with respect to Gu parameter'''
        Gu_grad  = {} 
        for user in self.Gu[e]:   
            total = np.zeros((1,self.k)) 
            for item in self.Iu[user]:
                if self.eui[user][item] == e:
                    total+= np.multiply((self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]),\
                    self.Gi[e][item])
            total /= self.Ntrain
            total += self.Lambda1*self.Gu[e][user]
            if e == self.E-1:
                total += self.Lambda2* (self.Gu[e][user] - self.Gu[e-1][user]) 
            elif e==0:
                total += self.Lambda2* (self.Gu[e][user] - self.Gu[e+1][user]) 
            else:
                total += self.Lambda2* (2*self.Gu[e][user] - self.Gu[e-1][user]  - self.Gu[e+1][user])
            Gu_grad[user] = total.copy()
        return Gu_grad
    
    def compute_gradient_wrt_Gi(self,e):
        ''' Compute gradient of objective with respect to Gi parameter'''
        Gi_grad = {}
        for item in self.Gi[e]:
            total = np.zeros((1,self.k))
            for user in self.Ui[item]:
                if self.eui[user][item] == e:
                    total+= np.multiply((self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]),\
                                        self.Gu[e][user])
            total /= self.Ntrain
            total += self.Lambda1*self.Gi[e][item]
            if e == self.E-1:
                total += self.Lambda2* (self.Gi[e][item] - self.Gi[e-1][item])
            elif e==0:
                total += self.Lambda2* (self.Gi[e][item] - self.Gi[e][item]) 
            else:
                total += self.Lambda2* (2*self.Gi[e][item] - self.Gi[e-1][item]  - self.Gi[e+1][item])
            Gi_grad[item] = total.copy()
        return Gi_grad
    
        
    def deepish_copy(self,org):
        '''much, much faster than deepcopy, for a dict of the simple python types.'''
        out = dict().fromkeys(org)
        for k,v in org.items():
            try:
                out[k] = v.copy()   # dicts, sets
            except AttributeError:
                try:
                    out[k] = v[:]   # lists, tuples, strings, unicode
                except TypeError:
                    out[k] = v      # ints

        return out  

In [13]:
lfmObj1 = ExpertiseLFM(train,Iu, Ui,quickmap,distinct_user_set,distinct_item_set,user_map_int, item_map_int,5,0.001,0.1,5,4)

Experience assignment done


In [43]:
np.savetxt("ratebeer_user_expertise_lfm_results_random", lfmObj1.final_param)
#np.savetxt("beeradvocate_user_expertise_lfm_results_lastrating", lfmObj1.final_param)
#np.savetxt("beeradvocate_user_expertise_lfm_results_random", lfmObj2.final_param)

In [45]:
eui_dict = {}
for user in lfmObj1.eui:
    for item in lfmObj1.eui[user]:
        eui_dict[user+'<?>'+item] = int(lfmObj1.eui[user][item])

with open("ratebeer_random_eui.json",'w') as outfile:
#with open("beeradvocate_lastrating_eui.json",'w') as outfile:
        json.dump(eui_dict,outfile)
    #json.dump(lfmObj1.eui,outfile)
#print("File {} written".format("finefood_random_eui.json"))
outfile.close()